# Introducción

En el siguiente proyecto estaré desarrollando un modelo de aprendizaje para un banco que está experimentando una creciente tasa de abandono por parte de sus clientes cada mes.  Para el banco es más económico salvar a los clientes que permanecen, que atrer a nuevos. 

Mi objetivo es crear un modelo de clasificación que me permita predecir exitosamente si un cliente abandonará el banco pronto. Para cumplir con esa tarea, estaré trabajando con la base de datos proporcionada por el banco, la cual me permitirá evaluar el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Mi enfoque en cuanto a las predicciones de si un cliente abandonará o no, será disminuir el máximo posible de falsos negativos y falsos positivos; para lo cual me concentraré en la media armónica de las métricas de precisión y sensibilidad.

#### Tabla de contenido:

1. Inicialización: importar las librerías con las que trabajaré y cargar los datos en un DataFrame.
 - 1.1  Preparar los datos. 
2. Segmentar los datos en sets de entrenamiento, validación y prueba.
3. Comparar la calidad de diferentes modelos.
4. Elegir un modelo y comprobar su calidad con el conjunto de prueba.
5. Prueba de cordura en el modelo elegido.
6. Conclusiones generales.